In [1]:
import numpy as np
import cv2

In [2]:
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

In [3]:
lk_params = dict(winSize=(200, 200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [4]:
cap = cv2.VideoCapture(0)

ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# PTS TO TRACK
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts, None, **lk_params)
    
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    img = cv2.add(frame,mask)
    cv2.imshow("tracking",img)
    
    k= cv2.waitKey(30) & 0xFF
    if k == 27:
        break
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
cv2.destroyAllWindows()
cap.release()

TypeError: 'NoneType' object is not subscriptable

In [1]:
import cv2
import numpy as np

In [5]:
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

preImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    
    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(preImg,nextImg,None,0.5,3,15,3,5,1.2,0)
    print(flow)
    mag, ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    
    hsv_mask[:,:,0] = ang/2
    
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    cv2.imshow('frame',bgr)
    
    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break
    preImg = nextImg
cap.release()
cv2.destroyAllWindows()

[[[ 1.7409858e-04  2.0011497e-04]
  [ 1.4125700e-04  2.2497475e-04]
  [-4.1979438e-05  1.6222516e-04]
  ...
  [-1.2000386e-04  1.0846256e-04]
  [-9.2619826e-05  2.7701817e-05]
  [-8.5959102e-05 -1.7199553e-05]]

 [[ 1.8957496e-04  2.4406864e-04]
  [ 1.0156939e-04  2.9418754e-04]
  [-2.2897648e-04  2.6313096e-04]
  ...
  [-3.9462585e-04 -1.1555208e-04]
  [-3.4765620e-04 -1.7942047e-04]
  [-3.1655908e-04 -2.0568598e-04]]

 [[ 1.7486945e-04  1.9348493e-04]
  [ 1.7827411e-05  1.8849962e-04]
  [-3.2010599e-04  1.9330332e-04]
  ...
  [-8.1621978e-04 -3.5449417e-04]
  [-7.1407476e-04 -4.1324439e-04]
  [-6.0366967e-04 -4.0905896e-04]]

 ...

 [[-1.2389283e-02 -8.1239152e-04]
  [-3.2695074e-02  5.0865957e-03]
  [-7.0489720e-02  2.0775523e-02]
  ...
  [ 1.3766211e-01  1.1900881e-01]
  [ 1.4169349e-01  1.0428086e-01]
  [ 1.2993939e-01  8.4276982e-02]]

 [[-2.7381063e-03 -2.7286850e-03]
  [-6.2186443e-03  6.6928373e-04]
  [-1.5543883e-02  1.1830573e-02]
  ...
  [ 5.9328068e-02  7.2467111e-02]
  [ 